# Week 2: Linear Models

# Theory stuff

## Supervised learning: classification and regression
For now, we'll stick to supervised learning: the case of machine learning where we want to learn a function from input features $\vec{x}$ to some output $y$, for which we have a dataset of many examples of inputs and their associated "true" outputs $\{\vec{x}_i, y_i\}$.

Simple supervised learning problems are split into two cases.
**Regression** is the case where each $y_i$ is continuous, a real-valued number, and **classification** is the case where $y_i$ is discrete, one of a few classes.

## Loss functions
The loss function maps from the parameters $\theta$ of a model, and the set of examples (features and their true labels, $\{\vec{x_i}, y_i\}_i$), to a single value that represents how badly the model fits the data.
Minimizing the loss function is equivalent to finding the "optimum" parameter settings, in the sense that if our model was generating the data, it is the model with these parameters that is most likely to have generated the data (instead of a model structured the same way with any other parameters).

We can justify this kind of inversion using Bayes' theorem:
$$
p(\vec{\theta} | \text{data}) = \frac{p(\text{data} | \vec{\theta}) \cdot p(\vec{\theta})}{p(\text{data})} \propto p(\text{data} | \vec{\theta})
$$
Therefore maximizing the probability that our model was the one that generated the data is equivalent to finding the most probable parameters given the data, which is exactly what we want to do.
We can also compute the loss value on held-out values (a validation set) to see how well the model generalizes to unseen data.

To make learning feasible, we usually assume that the loss function is possible to express as an average of per-example loss functions:
$$ L_\text{total}(\{\vec{x_i}, y_i\}_i^n, \vec{\theta}) = \frac{1}{n} \sum_i^n L((\vec{x}_i, y_i), \vec{\theta}) $$

Either way, it's important that the loss function is differentiable so we can minimize it with gradient descent.

### Squared error
The (mean) squared error (MSE) is the "default" loss function for regression:
$$ L(y, \hat{y}) = (y - \hat{y})^2 $$
where $y$ is the true value to be predicted, and $\hat{y}$ is the model's prediction.

It penalizes the model for mistakes super-linearly, giving larger weight to big mistakes than small ones.
This can make MSE vulnerable to outliers.

### Cross-entropy
Cross-entropy loss (or "log loss") is the most common loss function for classification.

The output of a classification model is a vector of probabilities, one per class, that should sum up to one.
The $i$th probability represents how confident the model is that the input corresponds to class $i$.
In an information-theoretic sense, cross-entropy loss measures how different the probability distribution given by the model is from the empirically-measured distribution for that one example (0 probability for every class except the true one, which has probability 1).

As the model's confidence in the correct class approaches 1, the cross-entropy loss approaches 0.
As the model's confidence in the correct class approaches 0, the cross-entropy loss increases to infinity.

#### The binary case
In binary classification, your model outputs a single probability $p(x)$: the probability that the output label is 1 (which is one minus the probability that the label is 0).
The true label $y$ is 0 or 1, and the loss is
$$L(\vec{x}, y; \vec{\theta}) = - y \cdot \log (p) - (1 - y) \cdot \log (1 - p)
= \begin{cases} -\log(p), & \text{if } y = 0 \\ -\log(1-p), & \text{if } y = 1 \end{cases}
$$

#### The multi-class case
In multi-class classification (with multinomial cross-entropy loss), we turn the true label into a "one-hot encoding" -- a vector with one entry per possible class, that's zero everywhere except the true class.
For example, for a problem with $n=4$ classes and an example with correct label of 1, the one-hot encoding is 
$$p_\text{true}(y) = \begin{bmatrix}0 & 1 & 0 & 0 \end{bmatrix}$$
which defines a discrete "probability distribution" over possible labels that just assigns probability 1 to the correct label.
Our model outputs a vector of confidences, $p_\text{model}(y)$.
The loss is
$$L = -\sum_{i=0}^n p_\text{true}(y_i) \log (p_\text{model}(y_i))
= \begin{cases} - \log p_\text{model}(y_i) & \text{if } y = i \\ \vdots \end{cases}$$
Often this is computed by taking the dot product of the one-hot encoded label vector with the probability vector, then taking the log.

Note that this loss function only cares about what probability the model assigns to the correct class, not any of the other classes.
This can make computing cross-entropy loss more efficient (i.e. by not computing the probabilities assigned to any other class).

#### Aside: the other terms in the Bayes theorem derivation
There are two terms we didn't talk about.
$p(\text{data})$ is the _probability of seeing the data we did under the true data generatng process_, commonly called the "evidence", and it generally doesn't impact our model-building. 

The other term, $p(\theta)$ is much more interesting. 
It's our prior belief, represented as a probability distribution, about what kinds of parameters we expect to see.
In machine learning, we almost always express our priors through how the model is structured, how we preprocess (or augment) the data, and what kind of regularization we use.
Using neural networks (next week), for instance, expresses a strong prior that the function we want to learn is a _hierarchical composition of simple patterns_.

#### Aside: other loss functions
Lots of the time, other loss functions make sense to use.
It's problem-dependent.
For example, you might want to use dice-coefficient loss to deal with unbalanced classes in problems like classifying each pixel of an image (the "semantic segmentation" problem).
MSE and cross-entropy are good default choices, though.

## Stochastic and minibatch gradient descent
Since the loss over the entire dataset decomposes into an average of per-example losses, we can approximate the total loss with an average over a smaller number of examples:
$$L(\{\vec{x}_i, y_i\}; \vec{\theta}) = \frac{1}{n} \sum_i^n L((\vec{x}_i, y_i), \vec{\theta}) \approx \frac{1}{m} \sum_i^m L((\vec{x}_i, y_i), \vec{\theta})$$
where $m$ is the "batch size", or the number of examples used to approximate the true dataset loss in this case.

This is called "minibatch gradient descent" or "stochastic gradient descent" (SGD) (sometimes this term is restricted for $m=1$).
It can converge faster than full-scale gradient descent because often small batches are sufficient to approximate the average loss reasonably well, and the average direction of steps taken in the direction opposite the gradient of the average minibatch loss is the same as if you were using the dataset loss.
As a result, minibatch gradient descent is considered the best way to train most machine learning models.

Some considerations when picking a batch size:
 - Smaller batches provide a less-accurate estimate of the direction to move parameters in at each step
 - Larger batches take a longer time to compute
 - [There is evidence that the noise introduced from small batches is beneficial for reaching optima that generalize out-of-sample (geometrically, "broad" minima instead of "sharp" ones)](https://arxiv.org/abs/1609.04836)
 - [In some sense lowering the learning rate and increasing the batch size have the same effect](https://arxiv.org/abs/1711.00489)
 - Large batch sizes compute faster per-example on GPUs, making training the model overall faster (by reducing the frequency with which the GPU and CPU need to swap memory)
 - Large batches may not fit on the available memory (eg video RAM in a GPU)
 - Power-of-two batch sizes compute faster on GPUs (because of how "tensor cores" work)
 
Common batch sizes are anywhere from 4 to 64, sometimes higher.

## Linear regression
Linear regression is the simplest model we'll cover.
Linear regression of one variable involves as parameters a vector of weights $\vec{w}$ and a scalar bias $b$; given a vector of input features $\vec{x}$, the model predicts $\hat{y} = \vec{w}^T \vec{x} + b$.
The output variable is a linear combination of the inputs weighted by the weight vector, plus a bias that captures something like the mean output.

While linear algebra gives us a closed-form solution, in practice it's ineffient with large datasets.
Instead, learning $\vec{w}$ and $b$ with gradient descent on mean-squared error produces the same optimum, since the learning problem for linear regression is convex.

Linear regression can actually be used to regress against any handmade set of features computed from the data, so long as the output is expected to be a linear combination of the features.
For instance, we can fit a quadratic polynomial by computing the square of every input feature, and treating them as completely new features: 
$$\vec{x} = \begin{bmatrix} x_1 & x_2 & \cdots \end{bmatrix} \rightarrow  \begin{bmatrix} x_1 & x_2 & \cdots & x_1^2 & x_2^2 & \cdots \end{bmatrix}$$

## Logistic regression
Despite the name, logistic regression is the linear model for classification.
Linear regression on the probability that a particular class is right doesn't make much sense, because:
 - Individual class probabilities must be in the range [0, 1] and linear regression may produce values outside of this range
 - All class probabilities must sum to 1

Logistic regression models follow a few steps:
 1. For each class, perform linear regression to an "unnormalized probability" for that class. This value is called the **logit**, and it corresponds to the natural log of the odds (as opposed to probability) that the class is correct. This linear regression makes sense, since the log-odds can take any value from $-\infty$ to $\infty$.
 2. Convert the vector of log-odds to a valid probability distribution. In the one-variable case, this is done through the _logistic function_. In the multi-variable case, it means applying the _softmax function_.
 3. Train the model (usually using cross-entropy loss) on the computed probability distribution over classes.

#### One-variable case and the logistic function
In the one-variable case, $y \in \{0, 1\}$ and the model is as follows:

$$
\begin{align}
&l = \vec{\alpha}^T \vec{x} + b & \text{where $l$ is the logit, the log of the odds that $y=1$} \\
&p = \sigma(l) = \frac{e^l}{e^l + 1} & \text{where $\sigma$ is the logistic function and $p$ is the model probability that $y = 1$} \\ 
&L = -y \log(p) - (1 - y) \log (1 - p) & \text{where $L$ is the (cross-entropy) loss}
\end{align}
$$

The logistic function converts from log-odds to probability by exponentiating the logit (to undo the log), then computing the ratio of the odds (numerator) to the total odds (denominator).
![logistic function](https://upload.wikimedia.org/wikipedia/commons/8/88/Logistic-curve.svg)
Image credit: [Wikipedia article on logistic regression](https://en.wikipedia.org/wiki/Logistic_regression)

The logistic function...
 - Maps every input to the range \[0, 1\]
 - Is continuously increasing
 - "Saturates" (getting near-zero derivative) for high and low input values
 
#### Multivariable case and the softmax function
This can be generalized to the case of more than one variable.
In this case, there is one logit value per class, $l_i$, and so there is one linear regression per class.
Then the softmax function is applied, which converts the logits into probabilities:
$$ p_i = \frac{\exp(l_i)}{\sum_j \exp(l_j)}$$
It works exactly the same way as the logistic function, taking the ratio of the unnormalized probability of one class to the total of unnormalized probabilities.

The probabilities computed by the softmax function...
 - Are each individually in the range \[0, 1\]
 - Together sum to 1
 - Increase monotonically with their associated logit
 
Another interpretation of the softmax function is that it's a "softened" or "smoothed" version of the argmax function, which sets the largest element of a vector to 1 and all other elements to 0.
The softmax function just makes the largest element near 1 and all of the other elements near 0 -- the fact that it's "softened" allows it to be differentiable, and so we can use it for gradient descent.

## Backpropagation
TODO

# TensorFlow stuff

## Loading data
### Datasets
TODO

### Iterators
TODO

## Saving and restoring models
TODO

## TensorBoard
TODO

## Keras
TODO

### Sequential models
TODO

## Example: linear regression (TensorFlow)
TODO

## Example: linear regression (Keras)
TODO